sourced from: https://medium.com/vacatronics/3-ways-to-calibrate-your-camera-using-opencv-and-python-395528a51615

run before everything to callibrate camera

In [1]:
import cv2
import numpy as np
import pathlib
import os
import cv2.aruco as aruco

In [2]:
def save_coefficients(mtx, dist, path):
    '''Save the camera matrix and the distortion coefficients to given path/file.'''
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()

def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

In [21]:

# Create gridboard, which is a set of Aruco markers
# the following call gets a board of markers 5 wide X 7 tall
gridboard = aruco.GridBoard_create(
        markersX=3, 
        markersY=3, 
        markerLength=0.1, 
        markerSeparation=0.01, 
        dictionary=aruco.Dictionary_get(aruco.DICT_4X4_50))

# Create an image from the gridboard
img = gridboard.draw(outSize=(900, 900))
cv2.imwrite("test_gridboard.jpg", img)

# Display the image to us
cv2.imshow('Gridboard', img)
# Exit on any key
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
print(str(board))

<aruco_GridBoard 0x111be55d0>


In [51]:
def calibrate_chessboard(dir_path, image_format, square_size, width, height):
    '''Calibrate a camera using chessboard images.'''
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.
    images = pathlib.Path(dir_path).glob(f'*.png')
    #images = pathlib.Path(dir_path)
    # Iterate through all images
    for fname in images:
        img = cv2.imread(str(fname))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
        print(fname)
        print(ret)
       

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return [ret, mtx, dist, rvecs, tvecs]

In [52]:
# Parameters
IMAGES_DIR = '/Users/ivyli/149proj/homeChessboards'
IMAGES_FORMAT = '.png'
SQUARE_SIZE = 2.5
WIDTH = 5
HEIGHT = 8

# Calibrate 
ret, mtx, dist, rvecs, tvecs = calibrate_chessboard(
    IMAGES_DIR, 
    IMAGES_FORMAT, 
    SQUARE_SIZE, 
    WIDTH, 
    HEIGHT
)
# Save coefficients into a file
save_coefficients(mtx, dist, "calibration_chessboard.yml")

/Users/ivyli/149proj/homeChessboards/eef.png
False
/Users/ivyli/149proj/homeChessboards/sadge.png
False
/Users/ivyli/149proj/homeChessboards/uuf.png
False
/Users/ivyli/149proj/homeChessboards/oof.png
False
/Users/ivyli/149proj/homeChessboards/aaf.png
False
/Users/ivyli/149proj/homeChessboards/yyf.png
False
/Users/ivyli/149proj/homeChessboards/iif.png
False


error: OpenCV(4.5.4) /private/var/folders/j_/rmvg8zw52690s8m6b80w4jn80000gn/T/pip-install-l94g_e6x/opencv-contrib-python_cdd27c205f86421a91737e2d1866e827/opencv/modules/calib3d/src/calibration.cpp:3694: error: (-215:Assertion failed) nimages > 0 in function 'calibrateCameraRO'


In [28]:
def calibrate_aruco(dirpath, image_format, marker_length, marker_separation):
    '''Apply camera calibration using aruco.
    The dimensions are in cm.
    '''
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    arucoParams = aruco.DetectorParameters_create()
    board = aruco.GridBoard_create(3, 3, marker_length, marker_separation, aruco_dict)

    counter, corners_list, id_list = [], [], []
    img_dir = pathlib.Path(dirpath)
    first = 0
    # Find the ArUco markers inside each image
    for img in img_dir.glob(f'*.png'):
        print(str(img))
        image = cv2.imread(str(img))
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = aruco.detectMarkers(
            img_gray, 
            aruco_dict, 
            parameters=arucoParams
        )
        if first == 0:
            corners_list = corners
            id_list = ids
        else:
            corners_list = np.vstack((corners_list, corners))
            id_list = np.vstack((id_list,ids))
        first = first + 1
        counter.append(len(ids))

    counter = np.array(counter)
    # Actual calibration
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(
        corners_list, 
        id_list,
        counter, 
        board, 
        img_gray.shape, 
        None, 
        None 
    )
    return [ret, mtx, dist, rvecs, tvecs]

In [29]:

# Parameters
IMAGES_DIR = '/Users/ivyli/149proj/homeChessboards'
IMAGES_FORMAT = '.png'
# Dimensions in cm
MARKER_LENGTH = 6.5
MARKER_SEPARATION = 0.6

# Calibrate 
ret, mtx, dist, rvecs, tvecs = calibrate_aruco(
    IMAGES_DIR, 
    IMAGES_FORMAT,
    MARKER_LENGTH,
    MARKER_SEPARATION
)
# Save coefficients into a file
save_coefficients(mtx, dist, "calibration_aruco.yml")

# Load coefficients
mtx, dist = load_coefficients('calibration_aruco.yml')
original = cv2.imread('/Users/ivyli/149proj/homeChessboards/board1.png')
dst = cv2.undistort(img, mtx, dist, None, None)
cv2.imwrite('undist.jpg', dst)

/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.30.08 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.30.20 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.35.01 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.34.50 PM.png
/Users/ivyli/149proj/homeChessboards/board1.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.34.39 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.30.42 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.30.31 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.34.28 PM.png
/Users/ivyli/149proj/homeChessboards/Screen Shot 2021-11-27 at 9.30.54 PM.png


True

In [30]:
mtx

array([[2.03494379e+03, 0.00000000e+00, 3.06593327e+02],
       [0.00000000e+00, 5.44357684e+03, 4.28989549e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [31]:
dist

array([[ 1.93698822e+00, -1.25582056e+02, -4.20302872e-02,
         3.30306420e-02,  2.47023267e+03]])